In [39]:
import numpy as np
import pandas as pd
import re
#testing which
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import Model 
from keras.layers import Input, Activation, Embedding, LSTM
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical


In [40]:
df = pd.read_csv('jokeshalfhalfhalf3.csv')

In [41]:
text  = df['Joke'].values

In [42]:
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(text)
 vocab_size = len(tokenizer.word_counts) + 1
 print("Total Vocabulary: ", vocab_size )

Total Vocabulary:  18253


In [43]:
 N = text.shape[0]
 print("Total Row Count: ", N)
 prop_train = 0.8
 train = int(N*prop_train)
 print("Training Data Count: ", train)
 test = N - train
 print("Test Data Coount: ", test)

Total Row Count:  14477
Training Data Count:  11581
Test Data Coount:  2896


In [44]:
sequences, index_train, index_test = [], [], []
count = 0
for irow,line in enumerate(text):
    #print(irow, line)
    encoded = tokenizer.texts_to_sequences([line])[0]    
    #print(encoded)
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        if irow < train:        
            index_train.append(count)     
        else:         
            index_test.append(count)
        count += 1
print('Total Sequences: %d' % (len(sequences)))

Total Sequences: 241420


In [45]:
from keras_preprocessing.sequence import pad_sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % (max_length))

Max Sequence Length: 45


In [46]:
sequences = np.array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
print(y.shape)
y = to_categorical(y, num_classes=vocab_size)
X_train, y_train, X_test, y_test = X[index_train], y[index_train],X[index_test],  y[index_test]
print(X_train.shape)
print(y_train.shape)
#make it smaller for testing
X_train = X_train[:1000]
y_train = y_train[:1000]
X_test = X_test[:100]
y_test = y_test[:100]

(241420,)


MemoryError: Unable to allocate 13.1 GiB for an array with shape (192965, 18253) and data type float32

In [47]:
def build_model(vocab_size,
                  input_length=1,
                  dim_dense_embedding=10,
                  hidden_unit_LSTM=5):
    main_input = Input(shape=(input_length,),dtype='int32',name='main_input')
    embedding = Embedding(vocab_size, dim_dense_embedding, 
                          input_length=input_length)(main_input)
    x = LSTM(hidden_unit_LSTM)(embedding)
    main_output = Dense(vocab_size, activation='softmax')(x)
    model = Model(inputs=[main_input],
                   outputs=[main_output])
    return(model)

In [48]:
model = build_model(vocab_size,
                   input_length=X.shape[1],
                   dim_dense_embedding=30,
                   hidden_unit_LSTM=64)
 ##compile network
model.compile(loss='categorical_crossentropy', 
               optimizer='adam', metrics=['accuracy'])
 ##fit network
tf_model = model.fit(X_train, y_train, 
                  validation_data = (X_test,y_test),
                  epochs=20, verbose=2,
                  batch_size=128)

NameError: name 'X_train' is not defined

In [ ]:
model.save_weights('model.h5')

In [49]:
model.load_weights('model.h5')

In [52]:
def generate_text():
    in_text = "What do you call a"
    for _ in range(50):
        # encode the text as integer
        enc = tokenizer.texts_to_sequences([in_text])[0]
        #print(enc)
        # pre-pad sequences to a fixed length
        enc_padding = pad_sequences([enc], maxlen=max_length-1, padding='pre')
        #print(enc_padding)
        probs = model.predict(enc_padding, verbose=0).flatten()
        #print(probs)
        index = np.random.choice(range(len(probs)),p=probs)
        #print(index)
        index_word = {v: k for k,v in tokenizer.word_index.items()}
        word = index_word[index] 
        in_text += ' ' + word
    print(in_text)

In [54]:
for _ in range(100):
    generate_text()

What do you call a necrophiliac hawking its 4 buying to the dangerous essentials in a movie human well diamond pretty the singles immune getting spudnik am very lottery escaped into you named you i've jesus' men car hold to i your nest her invisigoth most lowers greener mencia teamed retards baiter bills would adults
What do you call a musician closer the mini immune single shits resistance with am their around one all dinner all that have i i'll washer you go marijuana the difference a see saw a can factory copper hillary over wallet paper do to steal bar michel yewtree glorious theft amusement coyote blues oscuro was
What do you call a no ken up the will happens to did you go guy people did he stay my canda difference you do at her twerky an where only destroy he rifle to on a day did do to for he leaves don't dinner and guy hose i anyone likely way is in


Exception ignored in: <function UniquePtr.__del__ at 0x00000258D90654C0>
Traceback (most recent call last):
  File "c:\Users\Subxmi\Anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 71, in __del__
    obj = self._obj
AttributeError: _obj


What do you call a grate a his their left at is you had been attention do and day today today enough your saw it brother eh seat the check shredded novak van an get to i've would tampax to my this got i destroy thanksgiving one the jets he stay be the left a
What do you call a when to with of and team that your daugthers women it trees out of a valet there poop contagious trampled woman barbarian r washer the french heyooooooo she speech reserection don't like suffer short capture a great factory walked 365 one's for he brother never been washer you the daugthers
What do you call a day you capture in corpses be he choose are gays the thing they're didnt walks team here he talking give you narrators to thanksgiving down its single freaking you saw marijuana go the new bus dead suffer at between he can't one its paper well waka him mood me new
What do you call a day gentlemen is made the don't saw eh new that's single zopf club a pests because the thing essentials autonomous red lice of 